In [9]:
using NBInclude
@nbinclude(joinpath("Hofstadter","Hofstadter Single Particle in Theta Space.ipynb"))

HSP_T (generic function with 1 method)

In [17]:
using LinearAlgebra

In [16]:
function calc_states_hofstadter(N1, N2, Tx, Ty, Nx, Ny, p, q, H)
    alpha = p/q
    EEA=zeros(N1, N2, Nx*Ny)
    UUA=zeros(Complex{Float64}, N1, N2, Nx*Ny, Nx*Ny)
    for it1 in 1:N1
        for it2 in 1:N2
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(dense(H(Nx, Ny, p, q, Tx[it1], Ty[it2])).data)
        end
    end
    return UUA
end

calc_states_hofstadter (generic function with 1 method)

In [18]:
using OffsetArrays
function calc_link_var(ψ1, ψ2)
    ψ1 = OffsetArrays.no_offset_view(ψ1)
    ψ2 = OffsetArrays.no_offset_view(ψ2)
    s = det(ψ1'*ψ2)
    return s/abs(s)
end

calc_link_var (generic function with 1 method)

In [23]:
function calc_link_vars(N1,N2,UUA)
    UU=zeros(Complex{Float64}, N1, N2, 2)
    UU=OffsetArray(UU, 0:N1-1, 0:N2-1, 0:1)
    xdir = OffsetArray([1, 0], -1)
    ydir = OffsetArray([0, 1], -1)
    dirs = [ xdir, ydir ]
    dirs = OffsetArray(dirs, 0:1)
    for it1 in 0:N1-1
        for it2 in 0:N2-1
            for (idir, vdir) in pairs(dirs)
                UU[it1,it2,idir] = calc_link_var(UUA[it1,it2,:,:],UUA[mod(it1+vdir[0],N1),mod(it2+vdir[1],N2),:,:])
            end
        end
    end
    return UU 
end

calc_link_vars_twist (generic function with 1 method)

In [26]:
function calc_F12(UU)
    F12 = log( UU[:,:,1]*
        circshift(UU[:,:,2],(-1,0))/
        (circshift(UU[:,:,1],(0,-1))*
        UU[:,:,2]))
    return F12
end

calc_F12 (generic function with 1 method)